In [1]:
from sympy import *
init_printing()

## Characteristic constants

In [2]:
c = [Matrix(x) for x in [(-1, 1), ( 0, 1), ( 1, 1), (-1, 0), ( 0, 0), ( 1, 0), (-1,-1), ( 0, -1), ( 1, -1)]]

In [3]:
c

⎡⎡-1⎤  ⎡0⎤  ⎡1⎤  ⎡-1⎤  ⎡0⎤  ⎡1⎤  ⎡-1⎤  ⎡0 ⎤  ⎡1 ⎤⎤
⎢⎢  ⎥, ⎢ ⎥, ⎢ ⎥, ⎢  ⎥, ⎢ ⎥, ⎢ ⎥, ⎢  ⎥, ⎢  ⎥, ⎢  ⎥⎥
⎣⎣1 ⎦  ⎣1⎦  ⎣1⎦  ⎣0 ⎦  ⎣0⎦  ⎣0⎦  ⎣-1⎦  ⎣-1⎦  ⎣-1⎦⎦

In [4]:
w = [Rational(*x) for x in [(1,36), (1,9), (1,36), (1,9), (4,9), (1,9), (1,36), (1,9), (1,36)]]

In [5]:
w

In [6]:
sum(w)

In [7]:
c_s = sqrt(Rational(1,3))
c_s

## Moments

In [8]:
u_x, u_y, rho, tau = symbols('u_x u_y rho tau')

In [9]:
f_next = symarray('f_next', 9)
f_next

array([f_next_0, f_next_1, f_next_2, f_next_3, f_next_4, f_next_5,
       f_next_6, f_next_7, f_next_8], dtype=object)

In [10]:
f_curr = symarray('f_curr', 9)
f_curr

array([f_curr_0, f_curr_1, f_curr_2, f_curr_3, f_curr_4, f_curr_5,
       f_curr_6, f_curr_7, f_curr_8], dtype=object)

In [11]:
u = Matrix([u_x, u_y])
u

⎡uₓ ⎤
⎢   ⎥
⎣u_y⎦

In [12]:
from sympy.codegen.ast import Assignment

In [13]:
moments = cse([
    Assignment(rho, sum(f_curr)),
    Assignment(u_x, sum([ (c_i*f_curr[i])[0] for i, c_i in enumerate(c) ]) / sum(f_curr)),
    Assignment(u_y, sum([ (c_i*f_curr[i])[1] for i, c_i in enumerate(c) ]) / sum(f_curr))
], optimizations='basic', symbols=numbered_symbols(prefix='m'))
moments

In [14]:
for expr in moments[0]:
    print("const float %s = %s;" % (expr[0], ccode(expr[1])))

const float m0 = f_curr_3 + f_curr_6;
const float m1 = f_curr_1 + f_curr_2;
const float m2 = f_curr_0 + f_curr_4 + f_curr_5 + f_curr_7 + f_curr_8 + m0 + m1;
const float m3 = 1.0/m2;
const float m4 = f_curr_0 - f_curr_8;


In [15]:
for i, expr in enumerate(moments[1]):
    print(ccode(expr))

rho = m2;
u_x = -m3*(-f_curr_2 - f_curr_5 + m0 + m4);
u_y = m3*(-f_curr_6 - f_curr_7 + m1 + m4);


## Equilibrium

In [16]:
f_eq = []

for i, c_i in enumerate(c):
    f_eq_i = w[i] * rho * (  1
                           + c_i.dot(u)    /    c_s**2
                           + c_i.dot(u)**2 / (2*c_s**4)
                           - u.dot(u)      / (2*c_s**2) )
    f_eq.append(f_eq_i)

f_eq

## Collision

In [17]:
collide = [ Assignment(f_next[i], f_curr[i] + 1/tau * ( f_eq_i - f_curr[i] )) for i, f_eq_i in enumerate(f_eq) ]
collide

In [18]:
collide = cse(collide, optimizations='basic')
collide

In [19]:
for expr in collide[0]:
    print("const float %s = %s;" % (expr[0], ccode(expr[1])))

const float x0 = 6*u_y;
const float x1 = 6*u_x;
const float x2 = pow(u_y, 2);
const float x3 = 3*x2;
const float x4 = pow(u_x, 2);
const float x5 = 3*x4;
const float x6 = x5 - 2;
const float x7 = x3 + x6;
const float x8 = x1 + x7;
const float x9 = 1.0/tau;
const float x10 = (1.0/72.0)*x9;
const float x11 = 6*x2;
const float x12 = x0 - x5 + 2;
const float x13 = (1.0/18.0)*x9;
const float x14 = -x3;
const float x15 = 9*pow(u_x + u_y, 2);
const float x16 = -x1;
const float x17 = x14 + 6*x4 + 2;


In [20]:
for i, expr in enumerate(collide[1]):
    print(ccode(expr))

f_next_0 = f_curr_0 - x10*(72*f_curr_0 + rho*(-x0 + x8 - 9*pow(-u_x + u_y, 2)));
f_next_1 = f_curr_1 - x13*(18*f_curr_1 - rho*(x11 + x12));
f_next_2 = f_curr_2 - x10*(72*f_curr_2 - rho*(x1 + x12 + x14 + x15));
f_next_3 = f_curr_3 - x13*(18*f_curr_3 - rho*(x16 + x17));
f_next_4 = f_curr_4 - 1.0/9.0*x9*(9*f_curr_4 + 2*rho*x7);
f_next_5 = f_curr_5 - x13*(18*f_curr_5 - rho*(x1 + x17));
f_next_6 = f_curr_6 - x10*(72*f_curr_6 + rho*(x0 - x15 + x8));
f_next_7 = f_curr_7 - x13*(18*f_curr_7 + rho*(x0 - x11 + x6));
f_next_8 = f_curr_8 - x10*(72*f_curr_8 + rho*(x0 + x16 + x7 - 9*pow(u_x - u_y, 2)));
